In [17]:
%cd /home/mrearle/repos/R2R-EnvDrop-ObjAttn/

/mnt-homes/kraken/mrearle/repos/R2R-EnvDrop-ObjAttn


# Imports

In [18]:
import sys
import torch

metadata_parser_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/metadata_parser'
if metadata_parser_path not in sys.path:
    sys.path.append(metadata_parser_path)

mattersim_path = '/home/mrearle/datasets/Matterport3DSimulator/build'
if mattersim_path not in sys.path:
    sys.path.append(mattersim_path)
    
src_path = '/home/mrearle/repos/R2R-EnvDrop-ObjAttn/r2r_src'
if src_path not in sys.path:
    sys.path.append(src_path)


In [19]:
from parse_house_segmentations import HouseSegmentationFile

# Parameters

In [20]:
from param import args

In [21]:
# Model args
args.name = 'agent_bt_val'
args.attn = 'soft'
args.train = 'validlistener'
args.angle_feat_size = 128
args.accumulateGrad = True
args.featdropout = 0.4
args.subout = 'max'
args.optim = 'rms'
args.lr = 1e-4
args.iters = 10
args.maxAction = 35

# Obj attn args
base = False
args.obj_attn_type = "connection"
args.max_obj_number = 32
args.obj_aux_task = not base
args.obj_aux_task_weight = 0.1
args.dataset = "R2R"
args.include_objs = not base
args.include_objs_lstm = False
args.reduced_envs = True
args.buffer_objs = False

# Setup Agent

## Imports

In [22]:
from agent import Seq2SeqAgent
from collections import defaultdict
import json
import numpy as np
import h5py
from collections import OrderedDict
from train import setup, read_vocab, Tokenizer, TRAIN_VOCAB, read_img_features, R2RBatch, Evaluation

log_dir = args.log_dir
FEATURE_FILE = "/home/mrearle/storage/img_features/ResNet-152-imagenet.hdf5"

## Environments

In [23]:
setup()

vocab = read_vocab(TRAIN_VOCAB)
tok = Tokenizer(vocab=vocab, encoding_length=args.maxInput)

feat_h5 = h5py.File(FEATURE_FILE, "r")
obj_feat_h5 = h5py.File(args.OBJECT_FEATURES, "r")

featurized_scans = set(feat_h5.keys())

train_env = R2RBatch(
    feature_store=feat_h5, object_feat_store=obj_feat_h5, batch_size=args.batchSize, splits=["train"], tokenizer=tok
)

val_env_names = ["val_unseen"]

val_envs = OrderedDict(
        (
            (
                split,
                (
                    R2RBatch(
                        feature_store=feat_h5,
                        object_feat_store=obj_feat_h5,
                        batch_size=args.batchSize,
                        splits=[split],
                        tokenizer=tok,
                    ),
                    Evaluation([split], featurized_scans, tok),
                ),
            )
            for split in val_env_names
        )
    )

W2I is defaultdict True
OLD_VOCAB_SIZE 991
VOCAB_SIZE 992
VOACB 991
The feature size is 2048
Loading navigation graphs for 21 scans
R2RBatch loaded with 4895 instructions, using splits: train
The feature size is 2048
Loading navigation graphs for 11 scans
R2RBatch loaded with 2349 instructions, using splits: val_unseen


## Setup Agent

In [24]:
agent = Seq2SeqAgent(train_env, "", tok, args.maxAction)

agent_name = 'agent-obj(32)-aux(0.1)-reduced'

args.load = f'snap/{agent_name}/state_dict/best_val_unseen'
agent.results_path = f'results/{agent_name}.json'
print(
    "Loaded the listener model at iter %d from %s"
    % (agent.load(args.load), args.load),
    flush=True,
)

Using Bidir in EncoderLSTM
ObjAttention: Using class ConnectionwiseObjectAttention
Listener: Done Instantiating Model. Initializing Optimizers
Listener: Done Instantiating Optimizers. Initializing Loss
Listener: Done Instantiating Loss. Initializing Logs
Listener: Flushed
Listener: Done Instantiating Logs. Listener Initialized
Loaded the listener model at iter 82500 from snap/agent-obj(32)-aux(0.1)-reduced/state_dict/best_val_unseen


In [25]:
for env_name, (env, evaluator) in val_envs.items():
    agent.logs = defaultdict(list)
    agent.env = env

    iters = None
    agent.test(use_dropout=False, feedback="argmax", iters=iters)
agent_result = agent.write_results()



# Compare trajectory data

In [26]:
import json

In [27]:
ground_truth_path = 'tasks/R2R/data/R2R_val_unseen.json'
base_agent_path = 'results/agent_base-reduced.json'
obj_agent_path = 'results/agent-obj(32)-aux(0.1)-reduced.json'

In [31]:
with open(ground_truth_path) as f:
    ground_truth = json.load(f)
ground_truth = {v['path_id']: v['path'] for v in ground_truth}

with open(base_agent_path) as f:
    base_agent = json.load(f)
base_agent = {v['instr_id']: v['trajectory'] for v in base_agent}

with open(obj_agent_path) as f:
    obj_agent = json.load(f)
obj_agent = {v['instr_id']: v['trajectory'] for v in obj_agent}

In [41]:
succ_cases = []
fail_cases = []
both_succs = []
both_fails = []
for instr_id in base_agent:
    path_id, _ = instr_id.split('_')

    base_agent_success = base_agent[instr_id][-1][0] == ground_truth[int(path_id)][-1]
    obj_agent_success = obj_agent[instr_id][-1][0] == ground_truth[int(path_id)][-1]

    if not base_agent_success and obj_agent_success:
        succ_cases.append(instr_id)
    elif not obj_agent_success and base_agent_success:
        fail_cases.append(instr_id)
    elif base_agent_success and obj_agent_success:
        both_succs.append(instr_id)
    elif not base_agent_success and not obj_agent_success:
        both_fails.append(instr_id)

In [42]:
len(succ_cases), len(fail_cases), len(both_succs), len(both_fails)

(90, 79, 49, 2131)

In [47]:
succ_cases[0]

'5148_1'